![Logo TUBAF](https://github.com/nagelt/Teaching_Scripts/raw/9d9e29ecca4b04eaf7397938eacbf116d37ddc93/Images/TUBAF_Logo_blau.png)

Exercise material of the MSc-level course **Foundations of Soil Mechanics**.
Held at Technische Universität Bergakademie Freiberg.

Comments to:

*Prof. Dr. Thomas Nagel  
Chair of Soil Mechanics and Foundation Engineering  
Geotechnical Institute  
Technische Universität Bergakademie Freiberg.*

https://tu-freiberg.de/en/soilmechanics

# Erddruckberechnung -- Schwergewichtsmauer

Das folgende Beispiel ist an das Beispiel aus W. Fellin "Geotechnik -- Lernen mit Beispielen" angelehnt.

![Rankine.png](https://raw.githubusercontent.com/nagelt/Teaching_Scripts/master/Images/Schwergewichtsmauer.png)

**Aufgabe:** Berechnen Sie die wirkenden Erddruckkräfte und bestimmen Sie die Exzentrizität der Resultierenden der Sohlspannung der Schwergewichtsmauer. Die Wirkung des Erdwiderstands auf der Luftseite ist zu vernachlässigen.


In [120]:
import matplotlib.pyplot as plt
import numpy as np

#Some plot settings
import plot_functions.plot_settings

**Gegebene Größen in Bezug auf Geometrie und Hinterfüllmaterial:** $\alpha = 0°$, $\varphi = 32°$, $c = 0$, $\beta = 12°$, $\gamma = 19\,$kN/m³, $\delta_\text{a} = 2/3 \varphi$, $B_\text{b} = 2.6\,$m, $H_\text{b} = 1.2\,$m, $B_\text{bt} = 0.8\,$m, $H_\text{t} = 3.8\,$m, $B_\text{t} = 1\,$m, $p_\text{v} = 12\,$kN/m².

In [150]:
alpha = 0
phi = np.deg2rad(32)
beta = np.deg2rad(12)
delta = 2./3.*phi
gamma = 19.
c = 0

zmax = 3.8+1.2
pv = 10

B_b = 2.6
H_b = 1.2
B_bt = 0.8
H_t = 3.8
B_t = 1

Eddruckbeiwerte für die Anteile Eigengewicht und Flächenlast:

\begin{align*}
	K_\text{agh} &= \left[
	\frac{\cos(\varphi - \alpha)}{\cos \alpha \left( 1 + \sqrt{\displaystyle\frac{\sin(\varphi + \delta_\text{a}) \sin (\varphi - \beta)}{\cos(\alpha - \beta) \cos(\alpha + \delta_\text{a})}} \right)}
	\right]^2
	\\
	K_\text{ag} &= \frac{K_\text{agh}}{\cos (\alpha + \delta_\text{a})} 
    \qquad
    K_\text{agv} = K_\text{agh} \tan (\alpha + \delta_\text{a})
\end{align*}

$$
    K_\text{aph} = \frac{\cos \alpha \cos \beta}{\cos(\alpha - \beta)} K_\text{agh}
$$

In [143]:
K_agh = (np.cos(phi - alpha)/(np.cos(alpha) * (1 + np.sqrt(np.sin(phi+delta)*np.sin(phi-beta)/(np.cos(alpha+delta)*np.cos(alpha-beta))))))**2
K_agv = K_agh * np.tan(alpha+delta)
print("K_{agh} = %.3f" %K_agh)
print("K_{agv} = %.3f" %K_agv)

K_{agh} = 0.300
K_{agv} = 0.117


In [144]:
K_aph = np.cos(alpha)*np.cos(beta)/np.cos(alpha-beta)*K_agh
K_apv = K_aph * np.tan(alpha+delta)
print("K_{aph} = %.3f" %K_aph)
print("K_{apv} = %.3f" %K_apv)

K_{aph} = 0.300
K_{apv} = 0.117


Die Erdduckverteilungen ergeben sich nach

\begin{align}
    e_\text{ah} &= \gamma z K_\text{agh} + p_\text{v} K_\text{aph}
    \\
    e_\text{av} &= \gamma z K_\text{agv} + p_\text{v} K_\text{apv}
    \\
    e_\text{a} &= e_\text{ah} \frac{1}{\cos(\alpha + \delta_\text{a})}
\end{align}

Damit liegen am Fußpunkt der Wand folgende Werte vor:

In [145]:
e_ah = K_agh * gamma * zmax + K_aph * pv
e_av = K_agv * gamma * zmax + K_apv * pv
e_a = e_ah / np.cos(alpha+delta)

In [146]:
print("e_{ah} = %.3f kN/m²" %e_ah)
print("e_{av} = %.3f kN/m²" %e_av)
print("e_{a} = %.3f kN/m²" %e_a)

e_{ah} = 31.483 kN/m²
e_{av} = 12.296 kN/m²
e_{a} = 33.799 kN/m²


Die Resultierenden Erddruckkräfte ergeben sich aus der Annahme einer konstanten Verteilung der Auflastanteile und einer linearen Verteilung der Eigengewichtsanteile. Bspw. gilt für die Horizontalkomponente
$$
    E_\text{ah} = K_\text{aph} p_\text{v} z + K_\text{agh} \gamma \frac{z^2}{2}
$$

In [147]:
E_ah = K_aph*pv*z + K_agh * gamma * z * z/2
E_av = E_ah * np.tan(alpha+delta)
E_a = E_ah / np.cos(alpha+delta)

In [148]:
print("E_{ah} = %.2f kN/m" %E_ah)
print("E_{av} = %.2f kN/m" %E_av)
print("E_{a} = %.2f kN/m" %E_a)

E_{ah} = 86.20 kN/m
E_{av} = 33.67 kN/m
E_{a} = 92.55 kN/m


Die Wichte des Betons der Schwergewichtsmauer wird zu $\gamma_\text{B} = 24\,$kN/m³ angenommen.

In [130]:
gamma_B = 24

In [153]:
B_bt2 = B_b - B_t - B_bt
G = gamma_B * (H_b*B_b+H_t*B_t + H_t*B_bt2/2)
V = G + E_av
H = E_ah

In [154]:
print("V = %.2f kN/m" %V)
print("H = %.2f kN/m" %H)

V = 236.23 kN/m
H = 86.20 kN/m


In [155]:
delta_Q = np.arctan(H/V)
print("Die Resultierende der Sohlspannung ist um %.1f ° zur Vertikalen geneigt" %np.rad2deg(delta_Q))

Die Resultierende der Sohlspannung ist um 20.0 ° zur Vertikalen geneigt


In [158]:
M_H = gamma_B*(H_b*B_b*B_b/2+H_t*B_t*(B_bt+B_bt2+B_t/2) + H_t*B_bt2/2*(B_bt+B_bt2/3*2)) + E_av*B_b
print("Das haltende Moment um die Vorderkante der Stützmauer beträgt %.2f kNm/m" %M_H)

Das haltende Moment um die Vorderkante der Stützmauer beträgt 425.04 kNm/m


In [159]:
M_T = K_aph*pv*zmax * zmax/2 + K_agh * gamma * zmax * zmax/2 * zmax/3
print("Das treibende Moment der horizontalen Erddruckkraft um die Vorderkante der Stützmauer beträgt %.2f kNm/m" %M_T)

Das treibende Moment der horizontalen Erddruckkraft um die Vorderkante der Stützmauer beträgt 156.17 kNm/m


In [160]:
x_H = (M_H - M_T)/V
e = 1.3-x_H
print("Die Sohlspannungsresultierende greift mit einer Exzentrizität von %.2f m an." %e)

Die Sohlspannungsresultierende greift mit einer Exzentrizität von 0.16 m an.


## Weiterführende Aufgaben:

* Wie ändert sich der Wert, wenn man den Wandreibungswinkel zu 0 ansetzt?
* Welchen Einfluss hat die Neigung der Geländelinie?
* Greift die Kraft im Kern des Rechteckfundaments an?
* Ab welcher Auflast beginnt die Schwergewichtsmauer zu kippen, wenn $\delta_\text{a}=0$ angenommen wird?